In [ ]:
# Import libraries
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_diabetes
from pyBOWIE.core.main import BO

Usig Grid search utility from sklearn

In [ ]:
data = load_diabetes(return_X_y=True)
x_train, y_train = data

svr = GridSearchCV(
    svm.SVR(kernel="rbf", gamma=0.1), param_grid={"kernel": ["rbf", "sigmoid"], "C": np.logspace(0, 3, 4), "gamma": np.logspace(-2, 2, 5)},
    scoring="neg_root_mean_squared_error",
    cv = 5
)
svr.fit(x_train, y_train)
print(svr.best_params_, -svr.best_score_)

In [ ]:
def fit_svc_cross_val(x, data = load_diabetes(return_X_y=True), cv = 5):

    x_train, y_train = data
    size_x = x.shape[0]
    output = []
    
    for i in range(size_x):
        model = svm.SVR(C=x[i,0], gamma=x[i,1], kernel=x[i,2])
        output.append(-np.mean(cross_val_score(model, x_train, y_train, cv=cv, scoring="neg_root_mean_squared_error")))

    return np.array(output)

domain =[{'name': 'C',      'type': 'continuous', 'domain': (1.0, 1000.0)},
         {'name': 'gamma',  'type': 'continuous', 'domain': (1.e-02, 1.e+02)},
         {'name': 'kernel', 'type': 'categorical', 'domain': ("rbf", "sigmoid")},
         ]

In [ ]:
res = BO(fit_svc_cross_val, domain=domain, sense="minimize", verbose=1).optimize()

In [ ]:
res = BO(fit_svc_cross_val, domain=domain, sense="minimize", kern_discovery_evals = 4, n_jobs=-1, verbose=1).optimize()